## 캐글 자전거 수요 예측 : https://www.kaggle.com/competitions/bike-sharing-demand/overview
### 목표 : 전처리 방법 변경 및 모델을 Tensorflow 딥러닝 모델로 변경하여 제출 후 스코어 0.8 이하 도달하기

### baseline

In [44]:
!pip install xgboost

   ---------------------------------------- 0.0/150.0 MB ? eta -:--:--
   - -------------------------------------- 4.7/150.0 MB 26.4 MB/s eta 0:00:06
   --- ------------------------------------ 12.1/150.0 MB 30.3 MB/s eta 0:00:05
   ----- ---------------------------------- 19.7/150.0 MB 32.6 MB/s eta 0:00:04
   ------- -------------------------------- 26.7/150.0 MB 33.0 MB/s eta 0:00:04
   --------- ------------------------------ 34.9/150.0 MB 34.3 MB/s eta 0:00:04
   ----------- ---------------------------- 42.2/150.0 MB 34.4 MB/s eta 0:00:04
   ------------- -------------------------- 49.3/150.0 MB 34.4 MB/s eta 0:00:03
   --------------- ------------------------ 56.9/150.0 MB 34.9 MB/s eta 0:00:03
   ----------------- ---------------------- 64.7/150.0 MB 35.1 MB/s eta 0:00:03
   ------------------- -------------------- 72.9/150.0 MB 35.5 MB/s eta 0:00:03
   --------------------- ------------------ 80.7/150.0 MB 35.9 MB/s eta 0:00:02
   ------------------------ --------------- 90.2/1

In [270]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [271]:
# 데이터 불러오기 
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [272]:
# datetime 컬럼을 datetime 타입으로 변환
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])

# datetime에서 유용한 특성 추출
train['year'] = train['datetime'].dt.year
train['hour'] = train['datetime'].dt.hour
train['dayofweek'] = train['datetime'].dt.dayofweek
test['year'] = test['datetime'].dt.year
test['hour'] = test['datetime'].dt.hour
test['dayofweek'] = test['datetime'].dt.dayofweek

In [273]:
bins = [-np.inf, 10, 20, 30, np.inf]
labels = [0, 1, 2, 3]

train['temp_bin'] = pd.cut(train['temp'], bins=bins, labels=labels, right=False).cat.codes
test['temp_bin'] = pd.cut(test['temp'], bins=bins, labels=labels, right=False).cat.codes

train = train[train['count'] - train['count'].mean() < 3*train['count'].std()]
train = train.loc[train['weather'] !=4]

In [274]:
# 사용할 특성 선택
features = ['holiday', 'workingday', 'weather', 'temp_bin', 'season',
           'humidity', 'windspeed', 'year', 'hour']

X = train[features]
y = train['count']

In [275]:
# 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [276]:
# XGBoost 모델 학습
xgb_model = xgb.XGBRegressor(objective='count:poisson', n_estimators=300, random_state=42)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=300,
             n_jobs=None, num_parallel_tree=None, ...)

In [277]:
# 검증 데이터로 성능 평가
val_pred = xgb_model.predict(X_val)
print('검증 데이터 RMSE:', np.sqrt(mean_squared_error(y_val, val_pred)))
print('검증 데이터 MAE:', mean_absolute_error(y_val, val_pred))

검증 데이터 RMSE: 43.02343111607906
검증 데이터 MAE: 27.123476028442383


In [278]:
# 테스트 데이터 예측
X_test = test[features]
test_pred = xgb_model.predict(X_test)

In [279]:
# 제출 파일 생성
submission = pd.DataFrame({
    'datetime': test['datetime'],
    'count': test_pred
})
submission.to_csv('submission.csv', index=False)

print('제출 파일이 생성되었습니다.')

제출 파일이 생성되었습니다.
